In [1]:
from classifiers.inception import Classifier_INCEPTION


Using TensorFlow backend.


In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('full_data_shuffle_21_04(1167).csv')
data = data.drop(['Unnamed: 0'], axis = 1)
d_t = pd.DataFrame(StandardScaler().fit_transform(data.drop(['sample_num', 'gesture'], axis = 1)))
d_t['gesture'] = data['gesture']
d_t['sample_num'] = data['sample_num']
data = d_t
#data = data.sample(frac=1).reset_index(drop=True)
data.shape
len(data['sample_num'].unique())

1167

In [2]:
import numpy as np
np_lst = []
for n in data['sample_num'].unique():
    sample = data[data['sample_num'] == n].drop(['sample_num', 'gesture'], axis = 1)
    np_lst.append(sample.values)
    
data_reshaped = np.array(np_lst).reshape((1167,  14,100))
np_lst

[array([[-2.52565101e-01, -5.48193132e-02, -1.23021796e-01, ...,
          4.17281114e-01, -1.06458505e-01, -3.93926316e-02],
        [-1.88826267e-01, -6.05382526e-02, -4.05550566e-02, ...,
          5.14051149e-01, -3.46091199e-03,  1.52434759e-01],
        [-1.76944755e-01, -7.96234421e-03, -3.55149122e-02, ...,
          5.48504045e-01,  4.55436037e-02,  2.56506411e-01],
        ...,
        [ 8.87871926e-03, -7.53950141e-02, -2.60995751e-01, ...,
          6.02620551e-01,  2.46543749e-01,  2.27933665e-01],
        [ 3.58859856e-02, -8.62534213e-02, -2.10515554e-01, ...,
          5.96070279e-01,  1.76775157e-01,  1.95285919e-01],
        [ 2.33023405e-04, -1.15396388e-01, -3.19902439e-01, ...,
          3.89404077e-01,  6.38133994e-02, -5.97994621e-02]]),
 array([[ 0.01211454, -0.42570791,  0.2892594 , ...,  0.42548175,
         -0.08818223, -0.07205629],
        [-0.01597134, -0.42970627,  0.27579276, ...,  0.35987668,
         -0.09317079, -0.1414427 ],
        [ 0.01535878, -0.

In [3]:
from keras.utils import to_categorical
y = data['gesture']
l = []
for i in range(int(y.shape[0]/100)):
    if(y.values[i*100] == 'сжать руку'):
        l.append(0)
    else:
        l.append(1)
print(data['gesture'])      
y = to_categorical(np.array(l).reshape((len(l), )))
#y = np.array(l)

Using TensorFlow backend.


0           сжать руку
1           сжать руку
2           сжать руку
3           сжать руку
4           сжать руку
              ...     
116695    разжать руку
116696    разжать руку
116697    разжать руку
116698    разжать руку
116699    разжать руку
Name: gesture, Length: 116700, dtype: object


In [4]:
from sklearn.model_selection import StratifiedShuffleSplit

def shuffle_split(train_size = 0.9):
    sss = StratifiedShuffleSplit(n_splits = 1, test_size = 1-train_size)
    for train_index, test_index in sss.split(data_reshaped, y):
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = data_reshaped[train_index], data_reshaped[test_index]
        y_train, y_test = y[train_index], y[test_index]
    return X_train,  X_test, y_train, y_test
X_train, X_test, Y_train, Y_test = shuffle_split(0.95)

Y_train.shape
X_train.shape

TRAIN: [249 731 957 ...  10 749   3] TEST: [1025  885  330  784  439  215  996  667  214 1013  855  175  360   73
  370  494  304  443  823  110 1005  899   64 1128 1038  493 1157  345
 1166  163  265  471  325  660  136  611  120  586  453  287  652  626
  669   48  409  840  592  804  461  616  247 1079  824  904 1014  444
  943 1122  533]


(1108, 14, 100)

In [ ]:
def get_xp_val(xp):
    if xp == 'batch_size':
        xp_arr = [16, 32, 128]
    elif xp == 'use_bottleneck':
        xp_arr = [False]
    elif xp == 'use_residual':
        xp_arr = [False]
    elif xp == 'nb_filters':
        xp_arr = [16, 64]
    elif xp == 'depth':
        xp_arr = [3, 9]
    elif xp == 'kernel_size':
        xp_arr = [8, 64]
    else:
        raise Exception('wrong argument')
    return xp_arr


archive_name = 'TSC'
classifier_name = 'inception'
max_iterations = 5
output_directory = "."
input_shape = (14, 100)
nb_classes = 2

xps = ['use_bottleneck', 'use_residual', 'nb_filters', 'depth',
       'kernel_size', 'batch_size']
for xp in xps:

    xp_arr = get_xp_val(xp)

    print('xp', xp)

    for xp_val in xp_arr:
        print('\txp_val', xp_val)

        kwargs = {xp: xp_val}

        for iter in range(max_iterations):

            trr = ''
            if iter != 0:
                trr = '_itr_' + str(iter)
            print('\t\titer', iter)


            from classifiers import inception

            classifier = inception.Classifier_INCEPTION(output_directory, input_shape, nb_classes,
                                                                verbose=False, build=True, **kwargs, nb_epochs=5)

            classifier.fit(X_train, Y_train, X_test, Y_test, y)
            probs       = classifier.model.predict(X_test)
            preds       = probs.argmax(axis = -1)  
            acc         = np.mean(preds == Y_test.argmax(axis=-1))
            print("Classification accuracy: %f " % (acc))
            # the creation of this directory means
            create_directory(output_directory + '/DONE')

            print('\t\t\t\t', 'DONE')


xp use_bottleneck
	xp_val False
		iter 0
Instructions for updating:
If using Keras pass *_constraint arguments to layers.




In [5]:
def get_xp_val(xp):
    if xp == 'batch_size':
        xp_arr = [16, 32, 128]
    elif xp == 'use_bottleneck':
        xp_arr = [False]
    elif xp == 'use_residual':
        xp_arr = [False]
    elif xp == 'nb_filters':
        xp_arr = [16, 64]
    elif xp == 'depth':
        xp_arr = [3, 9]
    elif xp == 'kernel_size':
        xp_arr = [8, 64]
    else:
        raise Exception('wrong argument')
    return xp_arr


archive_name = 'TSC'
max_iterations = 5
output_directory = "C:\\Users\\ppkol\\Documents\\InceptionTime-master"
input_shape = (14, 100)
nb_classes = 2

xps = ['use_bottleneck', 'use_residual', 'nb_filters', 'depth',
       'kernel_size', 'batch_size']
for xp in xps:

    xp_arr = get_xp_val(xp)

    print('xp', xp)

    for xp_val in xp_arr:
        print('\txp_val', xp_val)

        kwargs = {xp: xp_val}

        for iter in range(max_iterations):

            trr = ''
            if iter != 0:
                trr = '_itr_' + str(iter)
            print('\t\titer', iter)


            from classifiers import nne

            classifier = nne.Classifier_NNE(output_directory, input_shape, nb_classes,
                                                                verbose=True)

            classifier.fit(X_train, Y_train, X_test, Y_test, y)
            probs       = classifier.model.predict(X_test)
            preds       = probs.argmax(axis = -1)  
            acc         = np.mean(preds == Y_test.argmax(axis=-1))
            print("Classification accuracy: %f " % (acc))
            # the creation of this directory means
            create_directory(output_directory + '/DONE')

            print('\t\t\t\t', 'DONE')


xp use_bottleneck
	xp_val False
		iter 0


OSError: Unable to open file (unable to open file: name = 'C:\Users\ppkol\Documents\InceptionTime-masterbest_model.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [8]:
clf = Classifier_INCEPTION(".", ( 14,100), 2, verbose=True, build=True, batch_size=16,
                 nb_filters=32, use_residual=True, use_bottleneck=True, depth=6, kernel_size=41, nb_epochs=60)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 14, 100)      0                                            
__________________________________________________________________________________________________
conv1d_33 (Conv1D)              (None, 14, 32)       3200        input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling1d_7 (MaxPooling1D)  (None, 14, 100)      0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_34 (Conv1D)              (None, 14, 32)       40960       conv1d_33[0][0]                  
____________________________________________________________________________________________

In [17]:
clf.fit( X_train, Y_train, X_test, Y_test, y, plot_test_acc=True)

NameError: name 'clf' is not defined

In [6]:
from classifiers import nne

classifier = nne.Classifier_NNE(output_directory, input_shape, nb_classes,
                                                    verbose=True)
classifier.fit(X_train, Y_train, X_test, Y_test, y)

OSError: Unable to open file (unable to open file: name = 'C:\Users\ppkol\Documents\InceptionTime-masterbest_model.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)